In [2]:

import os 
import time
import uuid
import threading
import cv2 as cv
import numpy as np
from mtcnn import MTCNN
import tensorflow as tf 
from ultralytics import YOLO 
import matplotlib.pyplot as plt 
from keras_facenet import FaceNet
from scipy.spatial.distance import euclidean


2024-10-09 23:48:28.027136: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 23:48:28.263856: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-09 23:48:28.332324: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-09 23:48:28.659919: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-09 23:48:32.187608: W tensorflow/compiler/tf2

In [3]:
embedding_database = []

In [4]:
embedding_database

[]

<br>

---
---
<br>

# `Collect Image With MTCNN`

<br>

---
---
<br>

In [3]:

# ---------------------------- Collecting the user Video ------------------------

fourcc = cv.VideoWriter_fourcc(*"XVID")
writer = cv.VideoWriter("test_video/data_collect.mp4", fourcc, 15.0, (780, 480))
cap = cv.VideoCapture(0)
cnt = 0

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv.flip(frame,1)
    cnt+=1
    if not ret:
        break
    
    # Resize frame if needed to match the VideoWriter's frameSize
    frame = cv.resize(frame, (780, 480))
    writer.write(frame)

    if cnt>0 and cnt<70:
        cv.putText(frame,"Look Forward",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
    
    if cnt>70 and cnt<100:
        cv.putText(frame,"Open Your Mouth",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
    if cnt>100 and cnt<130:
        cv.putText(frame,"Blink Your Eyes",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
    if cnt>130 and cnt<160:
        cv.putText(frame,"Look Left",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
    if cnt>160 and cnt<190:
        cv.putText(frame,"Look right",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
        
    if cnt>190 and cnt<220:
        cv.putText(frame,"Look Down",(50,250),fontFace=cv.FONT_HERSHEY_SIMPLEX,
                   fontScale=1,color=(0,255,0),
                   thickness=5)
    

    cv.imshow("COLLECT_DATA", frame)

    # Break loop on 'q' key press
    if cv.waitKey(1) & 0xFF == ord('q') or cnt==220:
        break

# Release resources
cap.release()
writer.release()
cv.destroyAllWindows()


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/yasin/allProgram/aurdino_project/venv/lib/python3.12/site-packages/cv2/qt/plugins"


In [4]:

# ------------------- collect image from the video ------------------------


# Initialize MTCNN detector
detector = MTCNN()

# Directory to save the images
DIR_NAME = "train_data"
# Path to the video file
path = "test_video/data_collect.mp4"

# Open the video file
cap = cv.VideoCapture(path)

# Function to perform face detection and draw rectangles
def detect_and_draw(frame):
    output = detector.detect_faces(frame)
    x, y, width, height = output[0]['box']
    cv.rectangle(img=frame, pt1=(x, y), pt2=(x + width, y + height), color=(255, 0, 0), thickness=5)
    modified_frame = frame[y:y+height, x:x+width]
    return frame, modified_frame

# Initialize variables for timing
last_processed_time = time.time()
processing_interval = 1  # Process every second

# Lock for thread-safe operations
lock = threading.Lock()
detected_frame = None  # Variable to hold the frame with detected faces
processing_frame = None
modified_frame = None  # Frame currently being processed by the thread

# Thread function for detection
def detection_thread():
    global processing_frame, detected_frame,modified_frame
    while cap.isOpened():
        if processing_frame is not None:
            with lock:
                # Run face detection and update the detected_frame
                detected_frame,modified_frame = detect_and_draw(processing_frame)
                processing_frame = None  # Reset processing frame once done

# Start the detection thread
thread = threading.Thread(target=detection_thread, daemon=True)
thread.start()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Get the current time
    current_time = time.time()
    
    # Check if 1 second has passed and no frame is currently being processed
    if current_time - last_processed_time >= processing_interval and processing_frame is None:
        last_processed_time = current_time
        with lock:
            processing_frame = frame.copy() 

    # If face detection was done, use the detected frame, otherwise use the original frame
    with lock:
        if detected_frame is not None:
            modified_frame, display_frame = modified_frame, detected_frame
        else:
            display_frame = frame

    # BGR to RGB, flip and show display
    display_frame = cv.cvtColor(display_frame, cv.COLOR_BGR2RGB)
    display_frame = cv.flip(display_frame, 1)

    # Save the modified frame
    if modified_frame is not None:
        unique_output_frame_name = "{}/{}.jpg".format(DIR_NAME, uuid.uuid1())
        cv.imwrite(unique_output_frame_name, modified_frame)

    # Show the output
    cv.imshow('output', display_frame)
    
    if cv.waitKey(1) == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv.destroyAllWindows()



qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/yasin/allProgram/aurdino_project/venv/lib/python3.12/site-packages/cv2/qt/plugins"


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


Exception in thread Thread-4 (detection_thread):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/home/yasin/allProgram/aurdino_project/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_69029/2772233847.py", line 40, in detection_thread
  File "/tmp/ipykernel_69029/2772233847.py", line 18, in detect_and_draw
IndexError: list index out of range


In [5]:
len(os.listdir("train_data"))

93

<br> 
<br>

---
---


# `Types of image we can get: `


### **Summary of the Conditions:**

1. **`len(img.shape) == 2`**:
   - This checks if the image is grayscale because grayscale images have no color channels (only height and width).
   - A grayscale image has the shape `(height, width)`.

2. **`img.shape[2] == 4`**:
   - This checks if the image has four channels (RGBA), where the fourth channel is the alpha (transparency) channel.
   - An RGBA image has the shape `(height, width, 4)`.

### Visual Example of Image Shapes:

1. **Grayscale Image:**
   ```python
   img.shape == (256, 256)  # len(img.shape) == 2
   ```

2. **RGB Image:**
   ```python
   img.shape == (256, 256, 3)  # img.shape[2] == 3 (Red, Green, Blue)
   ```

3. **RGBA Image:**
   ```python
   img.shape == (256, 256, 4)  # img.shape[2] == 4 (Red, Green, Blue, Alpha)
   ```

---

### How This Affects Preprocessing:
- **For grayscale images (`len(img.shape) == 2`)**, we need to convert them to RGB by adding color channels.
- **For RGBA images (`img.shape[2] == 4`)**, we need to remove the alpha channel, leaving only the RGB channels.


---
---

<br>
<br>



# Remember: For this project while collecting image: we should remember we only collect the rgb image: 
`Without RGB image we may get error: `

---
---

<br>
<br>




In [6]:

def preprocess_image(file_path):
    
    # read image as binary string 
    img = tf.io.read_file(file_path) 
        
    # as we are working with tensorflow we need to convert the binary string in image tensor
    # channels==3, ensuring it's an RGB image:
    img = tf.io.decode_image(img,channels=3) 
    
    """
    tf.io.decode_image does not automatically set a static shape for 
    the decoded image. TensorFlow needs a static shape to apply the tf.image.resize function.
    """
    
    # Set the shape of the image after decoding
    img.set_shape([None,None, 3])
    
    # resize image for keras_facenet:
    img = tf.image.resize(img,[160,160])
        
    # normalize the image[0 ~ 1]:
    img = tf.cast(img,tf.float64)/255.0
    
    return img

# ----------------------------------------- Create Dataset ----------------------------------

def create_dataset(image_folder_path, batch_size = 32):
    if not image_folder_path:
        raise ValueError("Not a valid Folder Path")
    
    # create a tensorflow dataset from the image_folder_path
    image_ds = tf.data.Dataset.list_files(f"{image_folder_path}/*")
    
    
    # map for using  preprocessing function:
    # image_ds = image_ds.map(preprocess_image,num_parallel_calls=tf.data.AUTOTUNE)
    # i will skip the value tf.data.AUTOTUNE because i don't have gpu
    image_ds = image_ds.map(preprocess_image)
    
    # select the batch size:
    image_ds.batch(batch_size=batch_size)
    
    # prefetch the performance:
    # skip this because i haven't cpu
    #image_ds = image_ds.prefetch(tf.data.AUTOTUNE)
    
    return image_ds


# ------------------------------------- Generate Embeddings ----------------

def generate_embedding(dataset):
    embeddings = []
    embedder = FaceNet()
    for batch in dataset:
        try: 
            embeddings_batch = embedder.embeddings([batch.numpy()])
            embeddings.append(embeddings_batch[0])
        
        except tf.errors.OutOfRangeError:
            print("Reached the end of the dataset.")
            break
        
        except Exception as e:
            print(f"Error during embedding generation:  {e}")
            continue
        
    avg_embeddings = np.mean(embeddings,axis=0)
    return avg_embeddings

# test the code: 
dataset = create_dataset('train_data')
store = generate_embedding(dataset=dataset)
if len(store)==0:
    print(" Embedding_store is not successfully. ")
    
else:
    print(store)
    embedding_database.append(store)
    print(" Embedding_store  successfully. ")



1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━

2024-10-09 23:50:22.721372: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<br>

---
---
<br>


# `Verify The Process: ` 




<br>
<br>

```python
box = [12.7, 34.9, 76.2, 90.5]  # Bounding box with float values
x1, y1, x2, y2 = map(int, box)   # Convert them to integers: [12, 34, 76, 90]

```
---
---
<br>

In [7]:
from numpy.linalg import norm

# Initialize MTCNN and FaceNet models
detector = MTCNN()
embedder = FaceNet()  # Using keras-facenet's FaceNet
print("MTCNN and FaceNet models initialized successfully.")

# Assume embedding_database is a list or array containing pre-stored embeddings
# For example, it could be loaded from a file or generated beforehand
# Here, we'll simulate it with a placeholder
# Replace this with your actual embedding loading mechanism
# Example: embedding_database = np.load('embeddings.npy')

if not embedding_database:
    raise ValueError("Embedding database is empty. Please load known embeddings.")
known_embedding = embedding_database[0]  # Use the first embedding as known

MTCNN and FaceNet models initialized successfully.


In [8]:



# Preprocess face for creating embedding with FaceNet
def preprocess(face_array):
    img = cv.resize(face_array, (160, 160))
    img = img.astype('float32')
    img = (img - 127.5) / 128.0  # Scaling to [-1, 1]
    return img

# Extract faces from the current video using MTCNN
def extract_faces(image, boxes):
    faces = []
    img_height, img_width = image.shape[:2]
    for box in boxes:
        x1, y1, width, height = box
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = x1 + width, y1 + height
        # Ensure coordinates are within the image
        x2 = min(x2, img_width)
        y2 = min(y2, img_height)
        face = image[y1:y2, x1:x2]
        if face.size == 0:
            continue  # Skip empty faces
        face = preprocess(face)
        faces.append(face)
    return faces

# Normalize the embedding
def normalize(embedding):
    norm_val = norm(embedding)
    return embedding / norm_val

# Verify faces by calculating the Euclidean distance between embeddings
def verify_faces(embedding1, embedding2, threshold=1.0):
    embedding1 = normalize(embedding1)
    embedding2 = normalize(embedding2)
    distance = norm(embedding1 - embedding2)
    print(f"Embedding Distance: {distance:.4f}")
    return distance < threshold

# =========================== Real-time video capture =============================================
def real_time_face_verification():
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        raise ValueError("Unable to open webcam. Please check your camera settings.")

    print("Starting real-time face verification. Press 'q' to exit.")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        # Convert frame to RGB as MTCNN expects RGB images
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

        # Detect faces using MTCNN
        results = detector.detect_faces(rgb_frame)

        if results:
            boxes = [result['box'] for result in results]
            faces = extract_faces(rgb_frame, boxes)

            if not faces:
                print("No valid faces extracted.")
                continue

           
            # keras-facenet expects a list of images
            embeddings = embedder.embeddings(faces)
            print(f"Generated {len(embeddings)} embeddings.")

            for i, embedding in enumerate(embeddings):
                print("known_embedding: {}".format(known_embedding))
                print("Current_Embedding: {}".format(embedding))
                is_same_person = verify_faces(known_embedding, embedding)

                if is_same_person:
                    x1, y1, width, height = boxes[i]
                    x2, y2 = x1 + width, y1 + height
                    # Draw rectangle and label on the original frame
                    cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv.putText(frame, "Yasin Arafat", (x1, y1 - 10),
                                cv.FONT_HERSHEY_SIMPLEX,
                                0.9, (0, 255, 0), 2)
                else:
                    x1, y1, width, height = boxes[i]
                    x2, y2 = x1 + width, y1 + height
                    # Optionally, label unknown faces
                    cv.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv.putText(frame, "Unknown", (x1, y1 - 10),
                                cv.FONT_HERSHEY_SIMPLEX,
                                0.9, (0, 0, 255), 2)

        # Display the frame with annotations
        cv.imshow("Real-Time Face Verification", frame)

        # Exit on pressing 'q'
        if cv.waitKey(1) & 0xFF == ord('q'):
            print("Exiting real-time face verification.")
            break

    cap.release()
    cv.destroyAllWindows()

# Entry point
if __name__ == "__main__":
    real_time_face_verification()


Starting real-time face verification. Press 'q' to exit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Generated 1 embeddings.
known_embedding: [   0.010408    0.013237   -0.081362    0.037037  -0.0058353    0.098731   -0.024931    0.066671    -0.03647   -0.026346   -0.016457    0.077464    0.029789   -0.036574  -0.0085121   -0.025437    0.019094    0.033401   -0.011352    -0.12971   -0.093507   -0.001864     0.07484   -0.042994    0.032512   -0.015244
   -0.016083    -0.04874   -0.043601    0.021088  -0.0094348 -0.00025411   0.0020879   -0.0236